Importación de librerías base para el problema 

In [22]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import os


os.getcwd()


'c:\\Users\\shado\\OneDrive\\Escritorio\\Latam\\LATAM'

##### Esta sección del codigo hace que los graficos dinamicos de pyplot queden estaticos para las vista en github.

In [23]:
import plotly.io as pio
pio.renderers
pio.renderers.default = "svg"

# 2.- Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :


#### Se inició por la segunda parte antes que el primer punto, ya que también analizare las variables agregadas externas al dataframe, por esta razón el punto 1 y 3 de análisis se estarán más abajo.

#### Lectura de datos principal y creacion de copia auxiliar de de la data original, también se busco información extra sobre países de los destinos de vuelos y los días feriados de ese año para el origen de los vuelos

#### Lectura y creación de variables en el data frame 

In [24]:
datos = pd.read_csv("dataset_SCL.csv", sep=",")
datos_original = datos.copy()
paises =  pd.read_excel("Prefijo Paises.xlsx", sheet_name= 'Hoja2')
feriados =  pd.read_excel("Feriados.xlsx")
datos.columns


C:\Users\shado\AppData\Local\Temp\ipykernel_59684\200450372.py:1: DtypeWarning:

Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.



Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'DIA', 'MES', 'AÑO', 'DIANOM', 'TIPOVUELO',
       'OPERA', 'SIGLAORI', 'SIGLADES'],
      dtype='object')

#### Verificación de tipos de datos en el data frame

In [25]:
datos.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

transformanción de Fechas a formato tiempo

In [26]:
datos["Fecha-I"] = pd.to_datetime(datos["Fecha-I"])
datos["Fecha-O"] = pd.to_datetime(datos["Fecha-O"])
datos.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

creación de variables dif_min, hora_dia, hora, minutos

In [27]:
datos['dif_min'] = (datos["Fecha-O"]- datos["Fecha-I"])
datos['hora_dia'] = datos['Fecha-I'].dt.time
datos['hora'] = datos['Fecha-I'].dt.hour
datos['minutos'] = datos['Fecha-I'].dt.minute

datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,-1 days +23:58:00,23:30:00,23,30


cambia la diferencia de las fechas de programación y salida de los vuelos a minutos.

In [28]:
datos["dif_min"] =  datos["dif_min"]/np.timedelta64(1,'m')

Homologación de variable Mes a nombre

In [29]:
conditions = [
    (datos['MES'] == 1),
    (datos['MES'] == 2),
    (datos['MES'] == 3),
    (datos['MES'] == 4),
    (datos['MES'] == 5),
    (datos['MES'] == 6),
    (datos['MES'] == 7),
    (datos['MES'] == 8),
    (datos['MES'] == 9),
    (datos['MES'] == 10),
    (datos['MES'] == 11),
    (datos['MES'] == 12)]
values = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre']
datos['Mes_Nombre'] = np.select(conditions, values)
datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos,Mes_Nombre
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,-2.0,23:30:00,23,30,Enero
